In [17]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model


import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

import tensorflow as tf
import pandas as pd
import keras

%matplotlib inline

In [2]:
def VGGONE(shape):
    X_input= Input(shape)
    X = X_input
    for i in range(4):
        X = Conv2D(filters=2**(i+5),kernel_size=(3,3),strides=(1,1),padding='same',name='conv2D_1_'+str(i))(X)
        X = BatchNormalization(axis=3,name='norm_1_'+str(i))(X)
        X = Activation('relu',name='active_1_'+str(i))(X)
        X = Conv2D(filters=2**(i+5),kernel_size=(3,3),strides=(1,1),padding='same',name='conv2D_2_'+str(i))(X)
        X = BatchNormalization(axis=3,name='norm_2_'+str(i))(X)
        X = Activation('relu',name='active_2_'+str(i))(X)
        X = MaxPooling2D(pool_size=(2,2),strides=(2,2),name='Max_pool'+str(i))(X)
    X = Flatten()(X)
    X = Dense(units=512,activation='relu',name='Dense1')(X)
    X = Dense(units=10,activation='softmax',name='ffc')(X)
    
    model = Model(inputs=X_input,outputs=X)
    
    return model


In [3]:
model = VGGONE([28,28,1])

In [4]:
model.get_config()

{'input_layers': [['input_1', 0, 0]],
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 28, 28, 1),
    'dtype': 'float32',
    'name': 'input_1',
    'sparse': False},
   'inbound_nodes': [],
   'name': 'input_1'},
  {'class_name': 'Conv2D',
   'config': {'activation': 'linear',
    'activity_regularizer': None,
    'bias_constraint': None,
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'bias_regularizer': None,
    'data_format': 'channels_last',
    'dilation_rate': (1, 1),
    'filters': 32,
    'kernel_constraint': None,
    'kernel_initializer': {'class_name': 'VarianceScaling',
     'config': {'distribution': 'uniform',
      'mode': 'fan_avg',
      'scale': 1.0,
      'seed': None}},
    'kernel_regularizer': None,
    'kernel_size': (3, 3),
    'name': 'conv2D_1_0',
    'padding': 'same',
    'strides': (1, 1),
    'trainable': True,
    'use_bias': True},
   'inbound_nodes': [[['input_1', 0, 0, {}]]],
   'name': 'conv2D_1_

In [5]:
df = pd.read_csv('train.csv')

In [6]:
label = df['label'].as_matrix()

In [7]:
label.shape

(42000,)

In [8]:
X = df.drop('label',axis=1).as_matrix()

In [9]:
X.shape

(42000, 784)

In [10]:
X1 = []
for i in range(X.shape[0]):
    X1.append(X[i,:].reshape(28,28,1))

In [11]:
resX = np.array(X1)

In [12]:
resX.shape

(42000, 28, 28, 1)

In [13]:
labels = []
with tf.Session() as sess:
    cl = tf.one_hot(label,10,axis=0)
    labels = sess.run(cl)

In [14]:
labels = labels.T

In [15]:
labels.shape

(42000, 10)

In [18]:
model.compile(optimizer=keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=["accuracy"])

In [19]:
model.compute_output_shape((4200,28,28,1))

(4200, 10)

In [20]:
model.fit(x=resX,y=labels,epochs=20,batch_size=32)

Epoch 1/20
42000/42000 [==============================] - 645s - loss: 0.1204 - acc: 0.9633   
Epoch 2/20
42000/42000 [==============================] - 642s - loss: 0.0496 - acc: 0.9853   
Epoch 3/20
42000/42000 [==============================] - 346s - loss: 0.0411 - acc: 0.9882   
Epoch 4/20
42000/42000 [==============================] - 338s - loss: 0.0340 - acc: 0.9899   
Epoch 5/20
42000/42000 [==============================] - 342s - loss: 0.0303 - acc: 0.9911   
Epoch 6/20
42000/42000 [==============================] - 348s - loss: 0.0263 - acc: 0.9925   
Epoch 7/20
42000/42000 [==============================] - 345s - loss: 0.0231 - acc: 0.9929   
Epoch 8/20
42000/42000 [==============================] - 338s - loss: 0.0193 - acc: 0.9944   
Epoch 9/20
42000/42000 [==============================] - 338s - loss: 0.0175 - acc: 0.9950   
Epoch 10/20
42000/42000 [==============================] - 336s - loss: 0.0158 - acc: 0.9953   
Epoch 11/20
42000/42000 [========================

In [21]:
model.save('VGGShort.h5')

In [16]:
dfU = pd.read_csv("test.csv")

In [17]:
inputA = dfU.as_matrix()

In [18]:
inputA.shape

(28000, 784)

In [19]:
X2 = []
for i in range(inputA.shape[0]):
    X2.append(inputA[i,:].reshape(28,28,1))

In [20]:
inputA = np.array(X2)

In [22]:
res = model.predict(inputA)

In [23]:
res.shape

(28000, 28, 28, 1)